In [55]:
import yfinance as yf
import seaborn as sns
import pandas as pd
import numpy as np

sns.set_theme(style="whitegrid", palette="viridis")

## Paskaičiuokite savo akcijos grąžą, riziką, maxdrawdown per metus nuo 2022-09-01 iki 2023-09-01. (1 balas)


In [56]:
from_date = "2022-09-01"
to_date = "2023-09-01"

symbol = "META"
stock = yf.Ticker(symbol)
hist = stock.history(start=from_date, end=to_date).dropna()

print(f"{symbol} Stats From {from_date} To {to_date}")

initial_price = hist["Close"].iloc[0]
final_price = hist["Close"].iloc[-1]

return_rate = (final_price / initial_price - 1) * 100
print(f"Return rate: {return_rate:.2f}%")

daily_returns = hist["Close"].pct_change().dropna()
volatility = daily_returns.std() * np.sqrt(daily_returns.size) * 100
print(f"Volatility: {volatility:.2f}%")


def calculate_max_drawdown(series):
    max_drawdown = 0
    peak = series.iloc[0]
    for value in series:
        if value > peak:
            peak = value
        else:
            drawdown = (peak - value) / peak
            max_drawdown = max(max_drawdown, drawdown)
    return max_drawdown


max_drawdown = calculate_max_drawdown(hist["Close"])
print(f"Maximum Drawdown: {max_drawdown:.2%}")

change = final_price - initial_price
return_percentage = (final_price / initial_price - 1) * 100
risk_free_percentage = 3
sharpe_ratio = (return_percentage - risk_free_percentage) / volatility
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

META Stats From 2022-09-01 To 2023-09-01
Return rate: 78.94%
Volatility: 53.33%
Maximum Drawdown: 47.44%
Sharpe Ratio: 1.42


## Prie savo gautos akcijos pridėkite dar keturias laisvai pasirinktas akcijas ir sumodeliuokite kelis portfelius, nurodykite kokią grąžą jie būtų patyrę 2022-09-01 iki 2023-09-01 laikotarpiu, kokia būtų rizika, Sharpe Ratio ir max drawdown:


In [57]:
start_date = "2022-09-01"
end_date = "2023-09-01"
initial_capital = 10000
stock_symbols = ["META", "MSFT", "GOOGL", "LMT", "GM"]

In [58]:
def get_stocks_info(symbol, start_date, end_date, risk_free_rate=3):
    stock = yf.Ticker(symbol)
    series = stock.history(start=start_date, end=end_date, actions=True, prepost=True)
    close_prices = series["Close"]

    daily_returns = close_prices.pct_change().dropna()
    volatility = daily_returns.std() * np.sqrt(daily_returns.size) * 100
    total_return = (close_prices.iloc[-1] / close_prices.iloc[0] - 1) * 100
    max_drawdown = get_max_drawdown(close_prices) * 100
    sharpe_ratio = (total_return - risk_free_rate) / volatility

    return {
        "symbol": symbol,
        "volatility": volatility,
        "return": total_return,
        "max_drawdown": max_drawdown,
        "sharpe_ratio": sharpe_ratio,
    }


def get_portfolio_values(stock_symbols, investment_distribution, start_date, end_date):
    data = {}
    portfolio_values = []

    for symbol in stock_symbols:
        stock = yf.Ticker(symbol)
        data[symbol] = stock.history(start=start_date, end=end_date)

    for i in range(len(data[stock_symbols[0]])):
        daily_value = sum(
            [
                data[symbol]["Close"].iloc[i]
                * (initial_capital * investment_distribution[symbol])
                / data[symbol]["Close"].iloc[0]
                for symbol in stock_symbols
            ]
        )
        portfolio_values.append(daily_value)

    return portfolio_values


def get_max_drawdown(series):
    max_drawdown = 0
    if isinstance(series, pd.Series):
        peak = series.iloc[0]
    if isinstance(series, list):
        peak = series[0]

    for value in series:
        if value > peak:
            peak = value
        else:
            drawdown = (peak - value) / peak
            max_drawdown = max(max_drawdown, drawdown)
    return max_drawdown


def get_volatility(portfolio_values):
    daily_returns = (
        np.array(portfolio_values[1:]) - np.array(portfolio_values[:-1])
    ) / np.array(portfolio_values[:-1])
    std_dev = np.std(daily_returns)
    annualized_volatility = std_dev * np.sqrt(252)
    annualized_volatility_percentage = annualized_volatility * 100

    return annualized_volatility_percentage


def get_sharpe_ratio(series, volatility, risk_free_rate=3):
    total_return = (series[-1] / series[0] - 1) * 100
    sharpe_ratio = (total_return - risk_free_rate) / volatility

    return sharpe_ratio


def get_some_stock_info(series):
    mdd = get_max_drawdown(series) * 100
    volatility = get_volatility(series)
    sharpe_ratio = get_sharpe_ratio(series, volatility)
    total_return = (series[-1] / series[0] - 1) * 100

    return {
        "return": total_return,
        "max_drawdown": mdd,
        "volatility": volatility,
        "sharpe_ratio": sharpe_ratio,
    }


def highest_total_return(stock_symbols, start_date, end_date):
    stock_returns = {}
    for symbol in stock_symbols:
        metrics = get_stocks_info(symbol, start_date, end_date)
        stock_returns[symbol] = metrics["return"]
        selected_stock = max(stock_returns, key=stock_returns.get)

    return selected_stock


def lowest_volatility(stock_symbols, start_date, end_date):
    stock_returns = {}
    for symbol in stock_symbols:
        metrics = get_stocks_info(symbol, start_date, end_date)
        stock_returns[symbol] = metrics["volatility"]

        selected_stock = min(stock_returns, key=stock_returns.get)
    return selected_stock


def highest_sharpe_ratio(stock_symbls, start_date, end_date):
    stock_returns = {}
    for symbol in stock_symbols:
        metrics = get_stocks_info(symbol, start_date, end_date)
        stock_returns[symbol] = metrics["sharpe_ratio"]

        selected_stock = max(stock_returns, key=stock_returns.get)
    return selected_stock

### 10000 tūkstančių eurų investuojama lygiomis dalimis į visas penkias akcijas (2 balai)


In [59]:
investment_distribution = {symbol: 0.20 for symbol in stock_symbols}
portfolio_a = get_portfolio_values(
    stock_symbols, investment_distribution, start_date, end_date
)

results_a = get_some_stock_info(portfolio_a)

pfa_df = pd.Series(results_a).to_frame().T
display(pfa_df)

,return,max_drawdown,volatility,sharpe_ratio
0,25.350992,16.945095,25.003249,0.893924


### 10000 investuojama į tą akciją, kuri 2017-09-01 iki 2022-09-01 laikotarpiu turėjo geriausią gražą. (1 balas)


In [60]:
investment_distribution = {}
selected_stock = highest_total_return(stock_symbols, "2017-09-01", "2022-09-01")
display(f"Selected stock {selected_stock}")
investment_distribution[selected_stock] = 1
portfolio_b = get_portfolio_values(
    [selected_stock], investment_distribution, start_date, end_date
)

results_b = get_some_stock_info(portfolio_b)

pfb_df = pd.Series(results_b).to_frame().T
display(pfb_df)

'Selected stock MSFT'

,return,max_drawdown,volatility,sharpe_ratio
0,27.083893,19.651221,31.252539,0.770622


### 10000 investuojama į tą akciją, kuri 2017-09-01 iki 2022-09-01 laikotarpiu turėjo mažiausią riziką. (1 balas)


In [61]:
investment_distribution = {}
selected_stock = lowest_volatility(stock_symbols, "2017-09-01", "2022-09-01")
display(f"Selected stock {selected_stock}")
investment_distribution[selected_stock] = 1
portfolio_c = get_portfolio_values(
    [selected_stock], investment_distribution, start_date, end_date
)

results_c = get_some_stock_info(portfolio_c)

pfc_df = pd.Series(results_c).to_frame().T
display(pfc_df)

'Selected stock LMT'

,return,max_drawdown,volatility,sharpe_ratio
0,8.870592,11.683054,20.542803,0.285774


### 10000 investuojama į tą akciją, kuri 2017-09-01 iki 2022-09-01 laikotarpiu turėjo geriausią Sharpe. (1 balas)


In [62]:
investment_distribution = {}
selected_stock = highest_sharpe_ratio(stock_symbols, "2017-09-01", "2022-09-01")
display(f"Selected stock {selected_stock}")
investment_distribution[selected_stock] = 1
portfolio_d = get_portfolio_values(
    [selected_stock], investment_distribution, start_date, end_date
)

results_d = get_some_stock_info(portfolio_d)

pfd_df = pd.Series(results_d).to_frame().T
display(pfd_df)

'Selected stock MSFT'

,return,max_drawdown,volatility,sharpe_ratio
0,27.083893,19.651221,31.252539,0.770622


### 10000 išskirstote į dvi akcijas, kurios turėjo mažiausią korealiaciją. 60% investuojate į tą akciją, kuri 2017-09-01 iki 2022-09-01 laikotarpiu turėjo didesnę grąžą ir 40% investuojate į kitą parinktą akciją, pagal korealiaciją. (2 balai)


In [63]:
def investment_strategy_e(start_date, end_date):
    investment_distribution = {}

    historical_data = {
        symbol: yf.Ticker(symbol).history(start=start_date, end=end_date)
        for symbol in stock_symbols
    }

    historical_closes = pd.DataFrame(
        {symbol: df["Close"] for symbol, df in historical_data.items()}
    )

    historical_returns = historical_closes.pct_change().dropna()
    correlation_matrix = historical_returns.corr()

    least_correlated_pair = correlation_matrix.unstack().idxmin()
    selected_stock = highest_total_return(least_correlated_pair, start_date, end_date)

    investment_distribution[selected_stock] = 0.60
    other_stock = (set(least_correlated_pair) - {selected_stock}).pop()
    investment_distribution[other_stock] = 0.40

    display(f"Portfolio E stocks: {selected_stock} and {other_stock}")

    portfolio_values = get_portfolio_values(
        least_correlated_pair, investment_distribution, start_date, end_date
    )

    return portfolio_values

In [64]:
portfolio_e = investment_strategy_e(start_date, end_date)
results_e = get_some_stock_info(portfolio_e)

pfe_df = pd.Series(results_e).to_frame().T
display(pfe_df)

'Portfolio E stocks: META and LMT'

,return,max_drawdown,volatility,sharpe_ratio
0,50.910361,22.833599,31.715233,1.510642


### e varianto investavimą naudojate iki 2023-06-01 ir likusiems tris mėnesiams savo kapitalą, kuris tuo metu yra, investuojate į: 75% į obligaciją, kuri turi 4% metinę grąžą, t.y. per šiuos tris mėnesius turės 1% grąžą. Kitus 25% investuojate į akcija, kuri 2018-06-01 iki 2023-06-01 turėjo mažiausią riziką. (2 balai)


In [65]:
investment_distribution = {}

portfolio_f = investment_strategy_e("2022-09-01", "2023-06-01")

current_portfolio_value = portfolio_f[-1]
initial_bond_investment = current_portfolio_value * 0.75
quarterly_interest_rate = 0.04 / 4
total_bond_return = initial_bond_investment * (1 + quarterly_interest_rate)
bond_profit = total_bond_return - initial_bond_investment

stock_investment = current_portfolio_value * 0.25
selected_stock = lowest_volatility(stock_symbols, "2018-06-01", "2023-06-01")
display(f"Lowest volatility stock {selected_stock}")
investment_distribution[selected_stock] = 0.25

new_portfolio_values = get_portfolio_values(
    [selected_stock], investment_distribution, "2023-06-01", "2023-09-01"
)

new_portfolio_values = [
    value + initial_bond_investment for value in new_portfolio_values
]
new_portfolio_values[-1] += bond_profit
portfolio_f.extend(new_portfolio_values)

results_f = get_some_stock_info(portfolio_f)

pff_df = pd.Series(results_f).to_frame().T
display(pff_df)

'Portfolio E stocks: META and LMT'

'Lowest volatility stock LMT'

,return,max_drawdown,volatility,sharpe_ratio
0,30.304423,22.833603,30.60165,0.892253
